In [10]:
import pandas as pd

In [11]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [91]:
plans = pd.read_csv('tests/data/simple.csv')
# remove dummy data
plans = plans[~plans.dpurp.isin(['dummy','Dummy'])]
plans.tail()

,pid,hid,tid,tseqno,hzone,ozone,dzone,dpurp,mdname,tstimei,tetimei,freq
6,2.0,0.0,6.0,1.0,a,a,b,work,car,500.0,530.0,10.0
7,2.0,0.0,7.0,2.0,a,b,a,work,car,800.0,830.0,10.0
8,2.0,0.0,8.0,3.0,a,a,c,work,car,1000.0,1030.0,10.0
9,2.0,0.0,9.0,4.0,a,c,a,work,car,1200.0,1230.0,10.0
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
from pam.population import Population, HouseHold, Person, Activity, Leg

In [101]:
population = Population()
for hid, household_data in plans.groupby('hid'):
    
    household = HouseHold(int(hid))
    
    for pid, person_data in household_data.groupby('pid'):
        
        trips = person_data.sort_values('tseqno')
        home_area = trips.hzone.iloc[0]
        current_activity = 'home'
        activity_map = {home_area: 'home'}
        activities = ['home']
        
        person = Person(int(pid), freq=person_data.freq.iloc[0])
        activity = Activity(1, 'home', home_area)
        person.add(Activity(1, 'home', home_area))
        
        for n in range(len(trips)):
            trip = trips.iloc[n]
            
            destination_activity = trip.dpurp
            destination_area = trip.dzone  # could assert here that origin is equal to previous destination
            
            person.add(
                Leg(
                    seq = n+1,
                    mode = trip.mdname,
                    start_loc=trip.ozone,
                    end_loc=trip.dzone,
                    start_time=trip.tstimei,
                    end_time=trip.tetimei
                )
            )
                
            if destination_activity in activities and activity_map.get(trip.dzone): # assume return trip to this activity
                person.add(Activity(n+2, activity_map[trip.dzone], trip.dzone))
            
            else:
                person.add(Activity(n+2, trip.dpurp, trip.dzone))
                
                if not trip.dzone in activity_map: # update history
                    activity_map[trip.dzone] = trip.dpurp # only keeping first activity at each location to ensure returns home        
                activities.append(destination_activity)
            
        household.add(person)
  
    population.add(household)

In [103]:
for seq, act in population.households[0].people[2].activities.items():
    print(act.act)

home
work
home
work
home
